# 連線Facebook爬蟲MongoDB資料庫，分析互動

In [1]:
import pymongo as mg
import datetime
import json
import plotly

from urllib.parse import quote
from plotly.graph_objs import *

In [2]:
plotly.__version__

'2.0.15'

# 每日發文數統計

In [3]:
keyword = ''
client = mg.MongoClient('mongodb://127.0.0.1:27017')
db_name = 'appledailytw'
db = client[db_name]

try:
    start_time = '2018-03-01 00:00:00'
    end_time = '2018-03-08 23:59:59'
    #query result
    #db.getCollection('post').find({status_published:{$gt: "2018-03-01 00:00:00",$lt: "2018-03-08 23:59:59"},status_message:{'$regex':''}})
    result = db['post'].find( { 'status_message': { '$regex': keyword },'status_published': {'$gt': start_time, '$lt': end_time}    }).sort("_id", -1)
    print ("Querying in database: %s" %(db_name))
    print ("total: %s records found." % result.count())

    result = list(result)
    temp_dict = {}
    date_dict = {}
    for x in result:
        temp_dict[ x['status_id'] ] = x
    print ("distinct %s records found." % (len(temp_dict) ) )

    post_dict = {}
    post_counter = 0
    last_sum = 0
    
    '''
    t_dic = {}
    for x in temp_dict:
        if temp_dict[x]["status_published"][:10] not in t_dic:
            t_dic[temp_dict[x]["status_published"][:10]] = 1
        else:
            t_dic[temp_dict[x]["status_published"][:10]] = t_dic[temp_dict[x]["status_published"][:10]]+1
    
    x_axis1 = []
    y_axis1 = []
    for x in t_dic:
        x_axis1.append(x)
        y_axis1.append(t_dic[x])
    zipped = zip(x_axis1, y_axis1)
    x_axis1 = []
    y_axis1 = []
    zipped = sorted(zipped, key=lambda x: x[0])
    for x in zipped:
        x_axis1.append(x[0])
        y_axis1.append(x[1])
    '''
    for x in temp_dict:
        last_sum = 0
        post_counter = 0
        #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
        dt = temp_dict[x]["status_published"][:10]
        try:
            last_sum = post_dict[dt]['counter']
            post_dict[dt]={'counter':last_sum+1}
        except Exception as e:
            post_counter = post_counter+1
            post_dict[dt]={'counter':post_counter}
            pass
    x_axis1 = []
    y_axis1 = []
    for x in post_dict:
        x_axis1.append(x)
        y_axis1.append(post_dict[x]['counter'])
    zipped = zip(x_axis1, y_axis1)
    x_axis1 = []
    y_axis1 = []
    zipped = sorted(zipped, key=lambda x: x[0])
    for x in zipped:
        x_axis1.append(x[0])
        y_axis1.append(x[1])
    #印出x軸List
    print(x_axis1)
    #印出y軸List
    print(y_axis1)
except Exception as e:
    print('Exception:'+str(e))
    pass

print('Done')

Querying in database: appledailytw
total: 584 records found.
distinct 584 records found.
['2018-03-01', '2018-03-02', '2018-03-03', '2018-03-04', '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08']
[35, 41, 22, 83, 125, 126, 121, 31]
Done


# 每日發文數統計視覺化

In [4]:
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
        "data": [{
                "x": x_axis1,
                "y": y_axis1
            }],
        "layout": {
            "title": "David 蘋果日報FB 每日發文數統計"
        }
    })

# 每日按讚統計

In [5]:
for x in temp_dict:
    last_sum = 0
    post_counter = 0
    #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["status_published"][:10]
    try:
        last_sum = post_dict[dt]['num_reactions']
        post_dict[dt]={'num_reactions':last_sum+temp_dict[x]['num_reactions']}
    except Exception as e:
        post_counter = post_counter+1
        post_dict[dt]={'num_reactions':post_counter}
        pass
#print(date_dict)
x_axis = []
y_axis = []
for x in post_dict:
    x_axis.append(x)
    y_axis.append(post_dict[x]['num_reactions'])

zipped = zip(x_axis, y_axis)
x_axis2 = []
y_axis2 = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis2.append(x[0])
    y_axis2.append(x[1])

#印出x軸、y軸
print(x_axis2)
print(y_axis2)

['2018-03-01', '2018-03-02', '2018-03-03', '2018-03-04', '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08']
[39495, 60854, 43611, 172471, 210163, 251948, 286566, 32092]


# 每日按讚統計視覺化

In [6]:
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
        "data": [{
                "x": x_axis2,
                "y": y_axis2
            }],
        "layout": {
            "title": "David 蘋果日報FB 每日按讚數統計"
        }
    })

# 每日按讚數 / 發文數統計視覺化

In [7]:
plotly.offline.init_notebook_mode(connected=True)
trace0 = Scatter(
    x=x_axis1,
    y=y_axis1,
    name = '每日發文篇數',
    line = dict(
        color = ('rgb(244, 67, 54)'),
        width = 4)
)
trace1 = Scatter(
    x=x_axis2,
    y=y_axis2,
    name = '每日按讚數',
    line = dict(
        color = ('rgb(33, 150, 243)'),
        width = 4)
)
data = Data([trace0, trace1])
plotly.offline.iplot({
        "data": data,
        "layout": {
            "title": "David 蘋果日報FB 每日發文/按讚數統計"
        }
    })

# 每日分享(share)統計

In [8]:
#分享
for x in temp_dict:
    last_sum = 0
    post_counter = 0
    #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["status_published"][:10]
    try:
        last_sum = post_dict[dt]['num_shares']
        post_dict[dt]={'num_shares':last_sum+temp_dict[x]['num_shares']}
    except Exception as e:
        post_counter = post_counter+1
        post_dict[dt]={'num_shares':post_counter}
        pass
#print(date_dict)
x_axis = []
y_axis = []
for x in post_dict:
    x_axis.append(x)
    y_axis.append(post_dict[x]['num_shares'])

zipped = zip(x_axis, y_axis)
x_axis3 = []
y_axis3 = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis3.append(x[0])
    y_axis3.append(x[1])
print(x_axis3)
print(y_axis3)

['2018-03-01', '2018-03-02', '2018-03-03', '2018-03-04', '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08']
[660, 4412, 2704, 4778, 4452, 5897, 7021, 618]


# 每日留言(comment)統計

In [9]:
#留言
for x in temp_dict:
    last_sum = 0
    post_counter = 0
    #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["status_published"][:10]
    try:
        last_sum = post_dict[dt]['num_comments']
        post_dict[dt]={'num_comments':last_sum+temp_dict[x]['num_comments']}
    except Exception as e:
        post_counter = post_counter+1
        post_dict[dt]={'num_comments':post_counter}
        pass
#print(date_dict)
x_axis = []
y_axis = []
for x in post_dict:
    x_axis.append(x)
    y_axis.append(post_dict[x]['num_comments'])

zipped = zip(x_axis, y_axis)
x_axis4 = []
y_axis4 = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis4.append(x[0])
    y_axis4.append(x[1])
print(x_axis4)
print(y_axis4)

['2018-03-01', '2018-03-02', '2018-03-03', '2018-03-04', '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08']
[3382, 3723, 3592, 7092, 9913, 10692, 12684, 1091]


# 每日分享(share) / 每日分享(留言) 比較視覺化

In [10]:
plotly.offline.init_notebook_mode(connected=True)
trace0 = Scatter(
    x=x_axis3,
    y=y_axis3,
    name = '每日分享數',
    line = dict(
        color = ('rgb(76, 175, 80)'),
        width = 4)
)
trace1 = Scatter(
    x=x_axis4,
    y=y_axis4,
    name = '每日留言數',
    line = dict(
        color = ('rgb(255, 87, 34)'),
        width = 4)
)
data = Data([trace0, trace1])
plotly.offline.iplot({
        "data": data,
        "layout": {
            "title": "David 蘋果日報FB 每日分享/留言數統計"
        }
    })

In [ ]:
# 回家作業：
# 按讚、留言、分享分析
# 粉絲頁：自己尋找
# 日期區間：自己定義
# 你的insight?